## Persistence

In [38]:
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
#db
# from langgraph.checkpoint.sqlite import SqliteSaver

model = ChatOllama(
    model="llama3.2:latest",
    temperature=0.3, 
    top_p=0.95, 
    num_ctx=2048, 
    repeat_penalty=1.2,
    timeout=120
)
#Define a new graph
workflow = StateGraph(state_schema=MessagesState)
#define a function that calls model
def call_model(state: MessagesState):
  response = model.invoke(state["messages"])
  return {"messages" : response}

#define a node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

#Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

config = {"configurable" : {"thread_id" : "thread_1"}}



## Here we told him our name

In [39]:

query = "Hi, I am bob"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)

output["messages"][-1].pretty_print()



================================== Ai Message ==================================

Hello Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?


## He remembers it

In [40]:
query = "What's my name?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)

output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is... Bob! We just established that earlier. How about we start fresh and see if we can have a fun conversation, huh?


## Here he should forget our name, since we changed the thread it

In [41]:
config = {"configurable" : {"thread_id" : "thread_2"}}
query = "What's my name?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don't know your name. I'm a large language model, I don't have the ability to store or recall information about individual users. Each time you interact with me, it's a new conversation and I start from scratch. If you'd like to share your name with me, I can use it to address you in our conversation!


In [42]:
config = {"configurable" : {"thread_id" : "thread_1"}}
query = "What's my name?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Déjà vu! Your name is still... Bob! I'm not sure how you forgot it already, but don't worry, I'll keep reminding you until you remember. Would you like to try and recall your own name or move on to something else?
